# Generative Adversarial Model

### Importing the required librarires

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
import numpy as np
import os
from torchvision.utils import save_image

#Flag if we need too create a new data array
REBUILD_DATA = True

#checking and setting the used device(either gpu or cpu)
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

### A class for reading all the images from the data directory. The images are read as grayscale images and are resized to 64X64 with OpenCV and appended to a numpy array. The array is saved in the trainingData.npy and can be directly loaded instead of always recreating the data.

In [ ]:
class Cats():
    label = "catData/data"
    
    trainingData = []
    catCount = 0
    
    def __init__(self):
        super().__init__()
        
    def make_training_data(self):
        for f in tqdm(os.listdir(self.label)):
            try:
                path = os.path.join(self.label, f)
                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (64, 64))
                self.trainingData.append([np.array(img)])

                self.catCount += 1
            except Exception as e:
                pass
        np.random.shuffle(self.trainingData)
        np.save("trainingData.npy", self.trainingData)
        print("CATS: " + str(self.catCount))

if REBUILD_DATA is True:   
    obj = Cats()
    obj.make_training_data()

In [ ]:
trainingData = np.load("trainingData.npy", allow_pickle = True)

### The two neural networks. In PyTorch a neural network could be a class. If implementing it as a class it inherits from nn.Module. The generator gets a z vector as input with a 100 features and generates an image. The descriminator has an image as input and outputs if the image is real or not.

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 64 * 64)
    
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.fc4(x)
        
        return torch.tanh(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(64 * 64, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        x = self.fc4(x)
        return torch.sigmoid(x)

#### Creating instances of the neural networks and moving them to the device(in my case the gpu).

In [ ]:
G = Generator().to(device)
D = Discriminator().to(device)

#### Setting the optimisers and the loss function. The Discriminator has to decide if the image is real or not and its optimiser has to decrease the error probability. The Generator has to produce images that can fool the Discriminator and its optimiser has to increase the Discriminator's error probability.

In [ ]:
criterion = nn.BCELoss()
genOptimizer = optim.Adam(G.parameters(), lr = 0.0002)
discOptimizer = optim.Adam(D.parameters(), lr = 0.0002)

#### Spling the data - 90% for training and 10% for testing. Also the data is converted from [0, 255] to [-1, 1].

In [ ]:
x = torch.Tensor([i[0] for i in trainingData]).view(-1, 1, 64, 64)
x = x / 255.0
x = (x - 0.5) / 0.5
VAL_PCT = 0.1
valSize = int(len(x) * VAL_PCT)
print(valSize)

In [ ]:
trainX = x[:-valSize]
testX = x[-valSize:]

Printing one image with the correct PyTorch method and matplotlib

In [ ]:
plt.imshow(trainX[0].view(64, 64), cmap = "gray")
plt.axis("off")
plt.show()

### Training function for the Discriminator. It is first given a batch of real images each of which is rearanged in a linear way(64X64 image will 1X784). The loss is calculated from the real amages then a batch of fake images is generated and tested on the neural network. The combined loss is summed and backpropagation is applied.

In [ ]:
def trainDiscriminator(x):
    D.zero_grad()

    # train discriminator on real
    xReal = x.view(-1, 64 * 64).to(device)
    dOutput = D(xReal)
    yReal = torch.ones(dOutput.size()[0], 1).to(device)
    dRealLoss = criterion(dOutput, yReal)
    dRealScore = dOutput

    # train discriminator on facke
    z = torch.randn(100, 100).to(device)
    xFake = G(z)
    yFake = torch.zeros(100, 1).to(device)

    dOutput = D(xFake)
    dFakeLoss = criterion(dOutput, yFake)
    dFakeScore = dOutput

    # gradient backprop & optimize ONLY D's parameters
    discLoss = dRealLoss + dFakeLoss
    discLoss.backward()
    discOptimizer.step()
        
    return  discLoss.data.item()

### Training function for the Generator. A batch of fake images is generated and tested on the Discriminator. The loss is calculated and backpropagation is applied.

In [ ]:
def trainGenerator(x):
    G.zero_grad()
    
    z = torch.randn(100, 100).to(device)
    y = torch.ones(100, 1).to(device)

    genOutput = G(z)
    discOutput = D(genOutput)
    genLoss = criterion(discOutput, y)

    # gradient backprop & optimize ONLY G's parameters
    genLoss.backward()
    genOptimizer.step()
        
    return genLoss.data.item()

### Training phase. TQDM is used to illustate the data loading better. On each EPOCH all the data is run through and the current statistics are printed. My dataset around 26000 images and it took 150 epochs to result in a relatively good image.

In [ ]:
EPOCHS = 150
BATCH = 100 
for epoch in range(EPOCHS):
    dLosses = []
    gLosses = []
    for i in tqdm(range(0, len(trainX), BATCH)):
        batchX = trainX[i : i + BATCH]
        dLosses.append(trainDiscriminator(batchX))
        gLosses.append(trainGenerator(batchX))
    print('[%d/%d]: loss disc: %.4f, loss gen: %.4f' % ((epoch + 1), EPOCHS, torch.mean(torch.FloatTensor(dLosses)), torch.mean(torch.FloatTensor(gLosses))))

### After training we can save the models and load them in another program just for testing purposes to simplify the code.

In [ ]:
torch.save(G.state_dict(), "linearGanGeneratorModel.pt")
torch.save(D.state_dict(), "linearGanDesciminatorModel.pt")

### Code to generate a new image with the generator and save it. Normalisation is used to return the image from [-1, 1] to [0, 255] which is used for grayscale images.

In [ ]:
with torch.no_grad():
    z = torch.randn(100, 100).to(device)
    generated = G(z)
    generated = generated.cpu()
    plt.imshow(generated[0].view(64, 64), cmap = "gray")
    plt.show()
    save_image(generated[0].view(64, 64), "sampleImage.png", normalize = True)